<a href="https://colab.research.google.com/github/tsenga2/keio-senga-seminar/blob/main/Kahle_Stulz_Table2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 100)

In [3]:
fileDownloaded = drive.CreateFile({'id':'118DvToqk7CY98eO_6Y7V6uzuF6jQid-N'})
fileDownloaded.GetContentFile('Table2.csv')

In [4]:
df = pd.read_csv('Table2.csv',encoding = "cp932")
df.head()

,日経会社コード,企業名称,決算期,決算種別,連結基準,<損益計算書（期首からの累計期間）>売上高・営業収益［累計］,<貸借対照表（資産）>資産合計,<貸借対照表（資産）>現金・預金／現金及び現金同等物,<損益計算書（期首からの累計期間）>経常利益／税金等調整前当期純利益［累計］,<基本項目>自己株式数,<基本項目>１株当たり配当金（各期末）,<基本項目>期末発行済株式総数
0,1,極洋,1975/10,10,0,95134.0,61804.0,10731.0,368.0,NaN,3.0,109633000.0
1,1,極洋,1976/10,10,0,114108.0,64419.0,12180.0,1433.0,NaN,4.0,109641000.0
2,1,極洋,1977/10,10,0,151255.0,76372.0,12479.0,2251.0,NaN,4.0,109642000.0
3,1,極洋,1978/10,10,0,165039.0,81093.0,15109.0,1269.0,NaN,3.0,109648000.0
4,1,極洋,1979/10,10,0,202157.0,90545.0,15776.0,616.0,NaN,3.0,110135000.0


In [5]:
df = pd.read_csv('Table2.csv',encoding = "cp932")
df = df.drop(columns={'企業名称'})
df = df.drop(columns={'決算種別'})
df = df.drop(columns={'連結基準'})
df = df.drop(columns={'<基本項目>自己株式数'})

In [6]:
df = df.rename(columns={'日経会社コード':'gvkey'})
df = df.rename(columns={'決算期':'fy'})
df = df.rename(columns={'<貸借対照表（資産）>資産合計':'assets'})
df = df.rename(columns={'<貸借対照表（資産）>現金・預金／現金及び現金同等物':'cash'})
df = df.rename(columns={'<損益計算書（期首からの累計期間）>売上高・営業収益［累計］':'earning'})
df = df.rename(columns={'<損益計算書（期首からの累計期間）>経常利益／税金等調整前当期純利益［累計］':'cash flow'})
df = df.rename(columns={'<基本項目>１株当たり配当金（各期末）':'dividends per'})
df = df.rename(columns={'<基本項目>期末発行済株式総数':'s number'})

In [7]:
fileDownloaded = drive.CreateFile({'id':'1cBelPYQ6tPFcK3OcACyR5x5FPp8MCugh'})
fileDownloaded.GetContentFile('Table2(consolidation).csv')

In [8]:
dff = pd.read_csv('Table2(consolidation).csv',encoding = "cp932")
dff = dff.rename(columns={'<基本項目>自己株式数':'s hold'})
dff = dff.rename(columns={'決算期':'fy'})
dff['fy'] = dff['fy'].str[:4]

In [9]:
df['dividends'] = df['dividends per'] * df['s number']
df['total assets'] = df['dividends'] + dff['s hold'] * df['dividends per']

In [10]:
df['fy'] = df['fy'].str[:4]

In [11]:
df.head()

,gvkey,fy,earning,assets,cash,cash flow,dividends per,s number,dividends,total assets
0,1,1975,95134.0,61804.0,10731.0,368.0,3.0,109633000.0,328899000.0,NaN
1,1,1976,114108.0,64419.0,12180.0,1433.0,4.0,109641000.0,438564000.0,NaN
2,1,1977,151255.0,76372.0,12479.0,2251.0,4.0,109642000.0,438568000.0,438572000.0
3,1,1978,165039.0,81093.0,15109.0,1269.0,3.0,109648000.0,328944000.0,NaN
4,1,1979,202157.0,90545.0,15776.0,616.0,3.0,110135000.0,330405000.0,NaN


In [12]:
df1 = df.groupby('fy').get_group('1975')
df1 = df1.fillna(0)

In [13]:
df1.head()

,gvkey,fy,earning,assets,cash,cash flow,dividends per,s number,dividends,total assets
0,1,1975,95134.0,61804.0,10731.0,368.0,3.0,109633000.0,328899000.0,0.0
41,2,1975,141362.0,136559.0,10236.0,-8307.0,0.0,163896000.0,0.0,0.0
77,3,1975,80224.0,111801.0,13303.0,165.0,2.5,200000000.0,500000000.0,501504850.0
78,3,1975,103903.0,126615.0,17238.0,1926.0,2.0,200000000.0,400000000.0,401350624.0
119,4,1975,412887.0,219746.0,17858.0,-1315.0,3.0,300000000.0,900000000.0,0.0


In [14]:
l1 = sorted(df1.loc[:,'earning'].to_list(),reverse=True)
total = l1[0]
n_1a = 0
while total <= sum(l1) * 0.5:
  n_1a += 1
  total += l1[n_1a]
print(f'n_1a={n_1a},total={total}')
n_1b = 100 * sum(l1[0:100]) / sum(l1)
print(f'n_1b={n_1b}')
print(l1[n_1a])

n_1a=65,total=48509382.0
n_1b=57.28091344176578
250371.0


In [15]:
l1 = sorted(df1.loc[:,'assets'].to_list(),reverse=True)
total = l1[0]
n_2a = 0
while total <= sum(l1) * 0.5:
  n_2a += 1
  total += l1[n_2a]
print(f'n_2a={n_2a},total={total}')
n_2b = 100 * sum(l1[0:100]) / sum(l1)
print(f'n_2b={n_2b}')

n_2a=72,total=67565556.0
n_2b=56.43709483235851


In [16]:
l1 = sorted(df1.loc[:,'cash'].to_list(),reverse=True)
total = l1[0]
n_3a = 0
while total <= sum(l1) * 0.5:
  n_3a += 1
  total += l1[n_3a]
print(f'n_3a={n_3a},total={total}')
n_3b = 100 * sum(l1[0:100]) / sum(l1)
print(f'n_3b={n_3b}')

n_3a=138,total=7109015.0
n_3b=43.233415328834226


In [17]:
l1 = sorted(df1.loc[:,'cash flow'].to_list(),reverse=True)
total = l1[0]
n_4a = 0
while total <= sum(l1) * 0.5:
  n_4a += 1
  total += l1[n_4a]
print(f'n_4a={n_4a},total={total}')
n_4b = 100 * sum(l1[0:100]) / sum(l1)
print(f'n_4b={n_4b}')

n_4a=29,total=933339.0
n_4b=78.269064116575


In [18]:
l1 = sorted(df1.loc[:,'dividends'].to_list(),reverse=True)
total = l1[0]
n_5a = 0
while total <= sum(l1) * 0.5:
  n_5a += 1
  total += l1[n_5a]
print(f'n_5a={n_5a},total={total}')
n_5b = 100 * sum(l1[0:100]) / sum(l1)
print(f'n_5b={n_5b}')

n_5a=79,total=281101685250.0
n_5b=54.327030853954575


In [19]:
l1 = sorted(df1.loc[:,'total assets'].to_list(),reverse=True)
total = l1[0]
n_6a = 0
while total <= sum(l1) * 0.5:
  n_6a += 1
  total += l1[n_6a]
print(f'n_6a={n_6a},total={total}')
n_6b = 100 * sum(l1[0:100]) / sum(l1)
print(f'n_6b={n_6b}')

n_6a=76,total=227838068197.72998
n_6b=55.09719823005891


In [20]:
df2 = df.groupby('fy').get_group('1995')
df2 = df2.fillna(0)

In [21]:
df2.head()

,gvkey,fy,earning,assets,cash,cash flow,dividends per,s number,dividends,total assets
20,1,1995,161431.0,56110.0,4534.0,635.0,0.0,113283000.0,0.0,0.0
61,2,1995,173152.0,95557.0,3963.0,2961.0,3.0,164492000.0,493476000.0,493773000.0
98,3,1995,400869.0,224873.0,7579.0,539.0,0.0,297012000.0,0.0,0.0
140,4,1995,420802.0,230828.0,3343.0,2413.0,0.0,300000000.0,0.0,0.0
178,5,1995,71860.0,39176.0,1589.0,844.0,0.0,60000000.0,0.0,0.0


In [22]:
l2 = sorted(df2.loc[:,'earning'].to_list(),reverse=True)
total = l2[0]
n_7a = 0
while total <= sum(l2) * 0.5:
  n_7a += 1
  total += l2[n_7a]
print(f'n_7a={n_7a},total={total}')
n_7b = 100 * sum(l2[0:100]) / sum(l2)
print(f'n_7b={n_7b}')

n_7a=91,total=231763813.0
n_7b=51.221756612192024


In [23]:
l2 = sorted(df2.loc[:,'assets'].to_list(),reverse=True)
total = l2[0]
n_8a = 0
while total <= sum(l2) * 0.5:
  n_8a += 1
  total += l2[n_8a]
print(f'n_8a={n_8a},total={total}')
n_8b = 100 * sum(l2[0:100]) / sum(l2)
print(f'n_8b={n_8b}')

n_8a=108,total=259045320.0
n_8b=48.37085670437315


In [24]:
l2 = sorted(df2.loc[:,'cash'].to_list(),reverse=True)
total = l2[0]
n_9a = 0
while total <= sum(l2) * 0.5:
  n_9a += 1
  total += l2[n_9a]
print(f'n_9a={n_9a},total={total}')
n_9b = 100 * sum(l2[0:100]) / sum(l2)
print(f'n_9b={n_9b}')

n_9a=125,total=23129419.0
n_9b=45.9248738808049


In [25]:
l2 = sorted(df2.loc[:,'cash flow'].to_list(),reverse=True)
total = l2[0]
n_10a = 0
while total <= sum(l2) * 0.5:
  n_10a += 1
  total += l2[n_10a]
print(f'n_10a={n_10a},total={total}')
n_10b = 100 * sum(l2[0:100]) / sum(l2)
print(f'n_10b={n_10b}')

n_10a=111,total=5771164.0
n_10b=47.5750462482379


In [26]:
l2 = sorted(df2.loc[:,'dividends'].to_list(),reverse=True)
total = l2[0]
n_11a = 0
while total <= sum(l2) * 0.5:
  n_11a += 1
  total += l2[n_11a]
print(f'n_11a={n_11a},total={total}')
n_11b = 100 * sum(l2[0:100]) / sum(l2)
print(f'n_11b={n_11b}')

n_11a=102,total=776943215750.0
n_11b=49.41650415972782


In [27]:
l2 = sorted(df2.loc[:,'total assets'].to_list(),reverse=True)
total = l2[0]
n_12a = 0
while total <= sum(l2) * 0.5:
  n_12a += 1
  total += l2[n_12a]
print(f'n_12a={n_12a},total={total}')
n_12b = 100 * sum(l2[0:100]) / sum(l2)
print(f'n_12b={n_12b}')

n_12a=82,total=458152093248.25
n_12b=54.83189364225138


In [28]:
df3 = df.groupby('fy').get_group('2015')
df3 = df3.fillna(0)

In [29]:
df3.head()

,gvkey,fy,earning,assets,cash,cash flow,dividends per,s number,dividends,total assets
40,1,2015,212086.0,73519.0,1635.0,2042.0,5.0,109282837.0,5.464142e+08,5.464192e+08
118,3,2015,350683.0,298391.0,141.0,5857.0,3.0,277210277.0,8.316308e+08,0.000000e+00
156,4,2015,414626.0,332052.0,1921.0,3486.0,30.0,52656910.0,1.579707e+09,1.579797e+09
225,6,2015,19556.0,17541.0,342.0,267.0,2.0,83790000.0,1.675800e+08,1.679203e+08
365,15,2015,46087.0,55694.0,6666.0,1172.0,2.0,88087784.0,1.761756e+08,1.761936e+08


In [30]:
l3 = sorted(df3.loc[:,'earning'].to_list(),reverse=True)
total = l3[0]
n_13a = 0
while total <= sum(l3) * 0.5:
  n_13a += 1
  total += l3[n_13a]
print(f'n_13a={n_13a},total={total}')
n_13b = 100 * sum(l3[0:100]) / sum(l3)
print(f'n_13b={n_13b}')

n_13a=102,total=198217904.0
n_13b=49.544915982992805


In [31]:
l3 = sorted(df3.loc[:,'assets'].to_list(),reverse=True)
total = l3[0]
n_14a = 0
while total <= sum(l3) * 0.5:
  n_14a += 1
  total += l3[n_14a]
print(f'n_14a={n_14a},total={total}')
n_14b = 100 * sum(l3[0:100]) / sum(l3)
print(f'n_14b={n_14b}')

n_14a=108,total=328230469.0
n_14b=48.263523983393654


In [32]:
l3 = sorted(df3.loc[:,'cash'].to_list(),reverse=True)
total = l3[0]
n_15a = 0
while total <= sum(l3) * 0.5:
  n_15a += 1
  total += l3[n_15a]
print(f'n_15a={n_15a},total={total}')
n_15b = 100 * sum(l3[0:100]) / sum(l3)
print(f'n_15b={n_15b}')

n_15a=121,total=23016412.0
n_15b=46.91699918557517


In [33]:
l3 = sorted(df3.loc[:,'cash flow'].to_list(),reverse=True)
total = l3[0]
n_16a = 0
while total <= sum(l3) * 0.5:
  n_16a += 1
  total += l3[n_16a]
print(f'n_16a={n_16a},total={total}')
n_16b = 100 * sum(l3[0:100]) / sum(l3)
print(f'n_16b={n_16b}')

n_16a=70,total=14632886.0
n_16b=56.34686643297412


In [34]:
l3 = sorted(df3.loc[:,'dividends'].to_list(),reverse=True)
total = l3[0]
n_17a = 0
while total <= sum(l3) * 0.5:
  n_17a += 1
  total += l3[n_17a]
print(f'n_17a={n_17a},total={total}')
n_17b = 100 * sum(l3[0:100]) / sum(l3)
print(f'n_17b={n_17b}')

n_17a=31,total=3646306994033.31
n_17b=64.87316729619614


In [35]:
l3 = sorted(df3.loc[:,'total assets'].to_list(),reverse=True)
total = l3[0]
n_18a = 0
while total <= sum(l3) * 0.5:
  n_18a += 1
  total += l3[n_18a]
print(f'n_18a={n_18a},total={total}')
n_18b = 100 * sum(l3[0:100]) / sum(l3)
print(f'n_18b={n_18b}')

n_18a=23,total=1464614340164.05
n_18b=77.44885485887364


In [36]:
list = [[n_1a,n_7a,n_13a,n_1b,n_7b,n_13b],[n_2a,n_8a,n_14a,n_2b,n_8b,n_14b],[n_3a,n_9a,n_15a,n_3b,n_9b,n_15b],
        [n_4a,n_10a,n_16a,n_4b,n_10b,n_16b],[n_5a,n_11a,n_17a,n_5b,n_11b,n_17b],[n_6a,n_12a,n_18a,n_6b,n_12b,n_18b]]
df = pd.DataFrame(list)
df.index = ['earning','assets','cash','cash flow','dividends','total assets']
df.columns=['1975','1995','2015','1975','1995','2015']
df

,1975,1995,2015,1975,1995,2015
earning,65,91,102,57.280913,51.221757,49.544916
assets,72,108,108,56.437095,48.370857,48.263524
cash,138,125,121,43.233415,45.924874,46.916999
cash flow,29,111,70,78.269064,47.575046,56.346866
dividends,79,102,31,54.327031,49.416504,64.873167
total assets,76,82,23,55.097198,54.831894,77.448855
